In [1]:
import sys
import os

src_path = os.path.abspath("../src")
if src_path not in sys.path:
    sys.path.insert(0, src_path)

In [2]:
import wikipedia
from hello_api.api_client import APIClient
import asyncio
import ollama 

SYSTEM_PROMPT = (
    "You are a helpful assistant helping a student prepare for a test. "
    "Take the following information and write detailed, accurate study notes."
)

async def generate_notes(subject: str) -> str:
    summary = wikipedia.summary(subject, sentences=3)

    response = await ollama.chat(
        model="llama3",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": f"Make detailed notes for: {subject}\n\n{summary}"}
        ]
    )
    return response["message"]["content"]


/Users/varshakumar/Documents/iMutoInternship/hello-fastapi-poetry/.venv/lib/python3.11/site-packages/pydantic/_internal/_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'orm_mode' has been renamed to 'from_attributes'
  warnings.warn(message, UserWarning)


In [3]:
from hello_api.notes.schema import NoteCreateUpdate

async def save_notes_to_db(subject: str, notes: str):
    client = APIClient()
    note = NoteCreateUpdate(title=subject, content=notes)
    await client.upsert_note(note)
    await client.close()


In [4]:
QUESTION_PROMPT = (
    "You are a test creator. Based on the following study notes, generate 10 DIFFICULT, factual, recall-based test questions. "
    "Only return the list of questions as strings."
)

async def generate_test_questions(notes: str) -> list[str]:
    response = ollama.chat(
        model="llama3",
        messages=[
            {"role": "system", "content": QUESTION_PROMPT},
            {"role": "user", "content": f"Generate questions based on:\n\n{notes}"}
        ]
    )
    questions = response["message"]["content"].strip().split("\n")
    return [q.strip("- ").strip() for q in questions if q.strip()]


In [5]:
import yaml

def save_test_to_yaml(test_dict: dict, file_path="testset.yaml"):
    with open(file_path, "w") as f:
        yaml.dump(test_dict, f)


In [6]:
async def run_full_pipeline(subjects: list[str]):
    test_results = {}
    for subject in subjects:
        print(f"Processing: {subject}")
        notes = await generate_notes(subject)
        await save_notes_to_db(subject, notes)
        questions = await generate_test_questions(notes)
        test_results[subject] = questions

    save_test_to_yaml(test_results)


In [ ]:
subjects = [
    "Voyager Program", "Chernobyl Disaster", "Periodic Table",
    "Human Genome Project", "French Revolution", "Apollo 11",
    "Great Barrier Reef", "Watergate Scandal", "CRISPR Gene Editing"
]

await run_full_pipeline(subjects)

Processing: Voyager Program
